In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

In [3]:
target = train.TARGET.values.copy()
train.drop('TARGET', axis=1, inplace=True)

In [4]:
train.loc[:, 'n0'] = (train == 0).sum(axis=1)
test.loc[:, 'n0'] = (test == 0).sum(axis=1)

In [5]:
dlist = list()
for c in train.columns:
    if train[c].unique().shape[0] == 1:
        print(c)
        dlist.append(c)
train.drop(dlist, axis=1, inplace=True)
test.drop(dlist, axis=1, inplace=True)

ind_var2_0
ind_var2
ind_var27_0
ind_var28_0
ind_var28
ind_var27
ind_var41
ind_var46_0
ind_var46
num_var27_0
num_var28_0
num_var28
num_var27
num_var41
num_var46_0
num_var46
saldo_var28
saldo_var27
saldo_var41
saldo_var46
imp_amort_var18_hace3
imp_amort_var34_hace3
imp_reemb_var13_hace3
imp_reemb_var33_hace3
imp_trasp_var17_out_hace3
imp_trasp_var33_out_hace3
num_var2_0_ult1
num_var2_ult1
num_reemb_var13_hace3
num_reemb_var33_hace3
num_trasp_var17_out_hace3
num_trasp_var33_out_hace3
saldo_var2_ult1
saldo_medio_var13_medio_hace3


In [6]:
for cone in train.columns:
    for ctwo in train.columns:
        if cone not in train.columns:
            continue
        if ctwo not in train.columns:
            continue
        if cone != ctwo and np.all(train[cone] == train[ctwo]):
            print(cone, ctwo)
            train.drop(ctwo, axis=1, inplace=True)
            test.drop(ctwo, axis=1, inplace=True)

ind_var6_0 ind_var29_0
ind_var6 ind_var29
ind_var13_medio_0 ind_var13_medio
ind_var18_0 ind_var18
ind_var26_0 ind_var26
ind_var25_0 ind_var25
ind_var32_0 ind_var32
ind_var34_0 ind_var34
ind_var37_0 ind_var37
ind_var40 ind_var39
num_var6_0 num_var29_0
num_var6 num_var29
num_var13_medio_0 num_var13_medio
num_var18_0 num_var18
num_var26_0 num_var26
num_var25_0 num_var25
num_var32_0 num_var32
num_var34_0 num_var34
num_var37_0 num_var37
num_var40 num_var39
saldo_var6 saldo_var29
saldo_var13_medio saldo_medio_var13_medio_ult1
delta_imp_reemb_var13_1y3 delta_num_reemb_var13_1y3
delta_imp_reemb_var17_1y3 delta_num_reemb_var17_1y3
delta_imp_reemb_var33_1y3 delta_num_reemb_var33_1y3
delta_imp_trasp_var17_in_1y3 delta_num_trasp_var17_in_1y3
delta_imp_trasp_var17_out_1y3 delta_num_trasp_var17_out_1y3
delta_imp_trasp_var33_in_1y3 delta_num_trasp_var33_in_1y3
delta_imp_trasp_var33_out_1y3 delta_num_trasp_var33_out_1y3


In [7]:
for c in train.columns:
    cmin = train[c].min()
    cmax = train[c].max()
    test.loc[test[c] < cmin, c] = cmin
    test.loc[test[c] > cmax, c] = cmax

In [8]:
import xgboost as xgb

In [9]:
weights = np.ones(target.shape)

In [10]:
weights[target == 1] = target.shape[0] / target.sum()

In [11]:
dtrain = xgb.DMatrix(train, target, weight=weights)

In [12]:
params = {'objective': 'binary:logistic', 
          'eval_metric': 'auc',
          'eta': 0.0202048,
          'max_depth': 5,
          'subsample': 0.6815,
          'colsample_bytree': 0.701,
          'silent': 1,
          'seed': 0
}

In [13]:
xgb.cv(params, dtrain, num_boost_round=560, nfold=5, stratified=True, verbose_eval=True)

[0]	cv-test-auc:0.7865172+0.044165035448417794	cv-train-auc:0.8044428+0.03280424928816386
[1]	cv-test-auc:0.8040001999999999+0.028295485119714768	cv-train-auc:0.8204376+0.02360768277150469
[2]	cv-test-auc:0.8086548+0.026297593854951846	cv-train-auc:0.8261010000000001+0.02429910182702234
[3]	cv-test-auc:0.8125432+0.02845345634821895	cv-train-auc:0.8310210000000001+0.025118148251811873
[4]	cv-test-auc:0.8144852+0.02884715273575539	cv-train-auc:0.8329912+0.024154725975676066
[5]	cv-test-auc:0.8142064+0.028763779199541897	cv-train-auc:0.833579+0.023589195492852235
[6]	cv-test-auc:0.822508+0.01649571498298876	cv-train-auc:0.840493+0.01313420439920136
[7]	cv-test-auc:0.8254085999999999+0.012323657129277818	cv-train-auc:0.8431198+0.00897294650379684
[8]	cv-test-auc:0.8281288+0.011194362373980925	cv-train-auc:0.8456954+0.006607541830363235
[9]	cv-test-auc:0.8297064000000001+0.010099125241326615	cv-train-auc:0.8470842+0.005487855624923109
[10]	cv-test-auc:0.829561+0.010846124137220643	cv-train-

KeyboardInterrupt: 

In [14]:
dtest = xgb.DMatrix(test)

In [15]:
gbm = xgb.train(params, dtrain, num_boost_round=256)

In [16]:
preds = gbm.predict(dtest)

In [17]:
preds

array([ 0.53241587,  0.61659712,  0.03572532, ...,  0.07458355,
        0.62134498,  0.02381196], dtype=float32)

In [18]:
sample = pd.read_csv('data/sample_submission.csv', index_col='ID')
sample.TARGET = preds
sample.to_csv('submission/init_xgb_w.csv', index_label='ID')